In [1]:
# Install necessary libraries
%pip install "pybamm[plot,cite]" -q

import pybamm
import matplotlib.pyplot as plt

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",  # for compatibility with older PyBaMM versions
    }
)

# Set model parameters
param = pybamm.ParameterValues("OKane2022")
param.update({
    "SEI solvent diffusivity [m2.s-1]": 2e-21,  # The new value can be adjusted as needed
    'Dead lithium decay constant [s-1]': 7.5e-5,
})

var_pts = {
    "x_n": 5,  # Negative electrode
    "x_s": 5,  # Separator
    "x_p": 5,  # Positive electrode
    "r_n": 30,  # Negative electrode particles
    "r_p": 30,  # Positive electrode particles
}

# Set the target number of cycles
target_cycle = 500

print(f"Calculating cycle {target_cycle} ...")

# Define the experimental protocol
exp_k = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
    + [
        (
            "Discharge at 1C until 2.5 V",
            "Charge at 1C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        )
    ]
    * target_cycle
    + ["Discharge at 0.1C until 2.5 V (5 minute period)"],
)

# Run the simulation
sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts)
sol_k = sim_k.solve()

# Extract LLI loss (unit: A.h) and calculate the remaining capacity
Q_LLI_final_k = sol_k["Total lithium lost [mol]"].entries[-1] * 96485.3 / 3600
remaining_capacity_k = (5 - Q_LLI_final_k) / 5 * 100

print(f"When k={target_cycle}, the remaining capacity is: {remaining_capacity_k:.2f}%")

Note: you may need to restart the kernel to use updated packages.
Calculating cycle 500 ...


At t = 469.698 and h = 9.84043e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 229.698 and h = 1.18976e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 404.83 and h = 1.08314e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 164.83 and h = 4.38548e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 399.889 and h = 1.27855e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 159.889 and h = 2.4656e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 395.035 and h = 1.26116e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 390.672 and h = 6.26748e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 386.499 and h = 5.62086e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 382.099 and h = 9.60107e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At 

When k=500, the remaining capacity is: 88.50%
